# No training required.

# Hugging Face gives you access to powerful models instantly.

# Used in real-world apps like customer support bots, review analysis, etc.

In [3]:
!pip install transformers torch --quiet

!pip install tf-keras --quiet

In [7]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

reviews = [
    "I love this product, it works perfectly!",
    "Worst quality ever, very disappointed.",
    "Absolutely amazing, exceeded my expectations!",
    "This is terrible, don’t buy it.",
    "Great quality and fast delivery!",
    "Not worth the money at all."
]

for r in reviews:
    result = classifier(r)[0]
    print(f"{r} ➝ {result['label']} ({result['score']:.2f})")

# Test
test = ["Amazing quality!", "Not good at all."]
for t in test:
    result = classifier(t)[0]
    print(f"{t} ➝ {result['label']} ({result['score']:.2f})")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


I love this product, it works perfectly! ➝ POSITIVE (1.00)
Worst quality ever, very disappointed. ➝ NEGATIVE (1.00)
Absolutely amazing, exceeded my expectations! ➝ POSITIVE (1.00)
This is terrible, don’t buy it. ➝ NEGATIVE (1.00)
Great quality and fast delivery! ➝ POSITIVE (1.00)
Not worth the money at all. ➝ NEGATIVE (1.00)
Amazing quality! ➝ POSITIVE (1.00)
Not good at all. ➝ POSITIVE (0.98)


🧠 Why “Not good at all” was classified as POSITIVE

This model is:

Pretrained mostly on movie reviews

Tuned for “POSITIVE” vs “NEGATIVE” on short phrases

Weak at negation phrases like “not good”

Doesn’t understand context as well as modern large models.

So:

"Amazing quality!" → ✅ POSITIVE (1.00)

"Not good at all." → ❌ incorrectly POSITIVE because "good" dominates the model’s embedding weighting more than "not".

In [ ]:
# Option 1: Use a better model

# Try a more robust sentiment model, e.g. cardiffnlp/twitter-roberta-base-sentiment

# This model has 3 labels: positive, neutral, and negative, and handles negation better.

from transformers import pipeline

classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

reviews = [
    "I love this product, it works perfectly!",
    "Worst quality ever, very disappointed.",
    "Absolutely amazing, exceeded my expectations!",
    "This is terrible, don’t buy it.",
    "Great quality and fast delivery!",
    "Not worth the money at all."
]

for r in reviews:
    result = classifier(r)[0]
    print(f"{r} ➝ {result['label']} ({result['score']:.2f})")

# Test
test = ["Amazing quality!", "Not good at all."]
for t in test:
    result = classifier(t)[0]
    print(f"{t} ➝ {result['label']} ({result['score']:.2f})")


Device set to use mps:0


I love this product, it works perfectly! ➝ LABEL_2 (0.99)
Worst quality ever, very disappointed. ➝ LABEL_0 (0.97)
Absolutely amazing, exceeded my expectations! ➝ LABEL_2 (0.97)
This is terrible, don’t buy it. ➝ LABEL_0 (0.94)
Great quality and fast delivery! ➝ LABEL_1 (0.72)
Not worth the money at all. ➝ LABEL_0 (0.88)
Amazing quality! ➝ LABEL_2 (0.56)
Not good at all. ➝ LABEL_1 (0.64)


In [8]:
# ✅ Install dependencies (only needed in Colab)
!pip install -q transformers torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
# Option 2: Preprocess the text

# Add rules before passing to the model:
# Lowercase
# Remove noise
# Handle negation with a simple rule (e.g., flag sentences with “not”, “never”, etc.)

# ✅ Import required libraries
from transformers import pipeline

# ✅ Load a pre-trained sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")

# ✅ Example product reviews (you can add more)
reviews = [
    "Amazing quality!",
    "Not good at all.",
    "I absolutely love it!",
    "This is terrible.",
    "Not good, I expected better.",
    "Awesome product and fast delivery!"
]

# ✅ Simple text preprocessing function
def preprocess(text):
    # Replace common negative phrases to make model more sensitive
    return text.lower().replace("not good", "bad")

# ✅ Classify each review after preprocessing
for r in reviews:
    cleaned = preprocess(r)
    result = classifier(cleaned)[0]
    print(f"{r} ➝ {result['label']} ({result['score']:.2f})")



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


Amazing quality! ➝ POSITIVE (1.00)
Not good at all. ➝ NEGATIVE (0.91)
I absolutely love it! ➝ POSITIVE (1.00)
This is terrible. ➝ NEGATIVE (0.99)
Not good, I expected better. ➝ POSITIVE (0.92)
Awesome product and fast delivery! ➝ POSITIVE (1.00)


What This Does

pipeline("sentiment-analysis") uses a pre-trained model (e.g., distilbert-base-uncased-finetuned-sst-2-english) for classifying text into POSITIVE or NEGATIVE.

The preprocess function normalizes text and adjusts phrases like “not good” to help the model capture negative sentiment better.

It prints the original review along with the predicted label and confidence score.

🧠 Why “Not good, I expected better.” got misclassified:

Pretrained sentiment models like distilbert-base-uncased-finetuned-sst-2-english are trained mostly on clean, short phrases (e.g., “good”, “bad”, “I love it”).

Phrases like “not good, I expected better” have mixed signals:

“not good” ➝ negative

“expected better” ➝ could be interpreted as neutral or slightly positive in some contexts

The model puts more weight on “expected better” and misclassifies it as POSITIVE.

In [10]:
# Option 3: Preprocess the text in a more advanced way

# Add rules before passing to the model:
# Lowercase
# Remove noise
# Handle negation with a simple rule (e.g., flag sentences with “not”, “never”, etc.)

# ✅ Import required libraries
from transformers import pipeline

# ✅ Load a pre-trained sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")

# ✅ Example product reviews (you can add more)
reviews = [
    "Amazing quality!",
    "Not good at all.",
    "I absolutely love it!",
    "This is terrible.",
    "Not good, I expected better.",
    "Awesome product and fast delivery!"
]

# ✅ Simple text preprocessing function
def preprocess(text):
    txt = text.lower()
    txt = txt.replace("not good", "bad")
    txt = txt.replace("expected better", "bad")
    return txt

# ✅ Classify each review after preprocessing
for r in reviews:
    cleaned = preprocess(r)
    result = classifier(cleaned)[0]
    print(f"{r} ➝ {result['label']} ({result['score']:.2f})")



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


Amazing quality! ➝ POSITIVE (1.00)
Not good at all. ➝ NEGATIVE (0.91)
I absolutely love it! ➝ POSITIVE (1.00)
This is terrible. ➝ NEGATIVE (0.99)
Not good, I expected better. ➝ NEGATIVE (0.93)
Awesome product and fast delivery! ➝ POSITIVE (1.00)
